In [1]:
#importing the required libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#loading the dataset
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("validation.csv")
test_df = pd.read_csv("test.csv")

#splitting the datframe into X and y
X_train = train_df['Message']
y_train = train_df['Label']
X_val = val_df['Message']
y_val = val_df['Label']
X_test = test_df['Message']
y_test = test_df['Label']

In [3]:
# get tf-idf values of training data

tfidf = TfidfVectorizer()

train_tfidf = tfidf.fit_transform(X_train)
train_tfidf

<3342x6674 sparse matrix of type '<class 'numpy.float64'>'
	with 44354 stored elements in Compressed Sparse Row format>

In [4]:
#get the tf-idf values of testing data
test_tfidf = tfidf.transform(X_test)
test_tfidf

<1115x6674 sparse matrix of type '<class 'numpy.float64'>'
	with 13927 stored elements in Compressed Sparse Row format>

In [5]:
#get the tf-idf values of validation data
val_tfidf = tfidf.transform(X_val)
val_tfidf

<1115x6674 sparse matrix of type '<class 'numpy.float64'>'
	with 13535 stored elements in Compressed Sparse Row format>

## Applying Naive Bayes

In [6]:
#use Naive Bayes classifier to fit the training data
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(train_tfidf, y_train)

#predict the results on test data
y_pred = naive_bayes_classifier.predict(test_tfidf)

#accuracy score of Naive Bayes
score1 = metrics.accuracy_score(y_test, y_pred)
print("The accuracy score of Naive Bayes is : ",score1)

#Confusion matrix of Naive Bayes

print(metrics.classification_report(y_test, y_pred, target_names=['Ham', 'Spam']))
print(metrics.confusion_matrix(test_df["Label"], y_pred))

The accuracy score of Naive Bayes is :  0.9587443946188341
              precision    recall  f1-score   support

         Ham       0.95      1.00      0.98       968
        Spam       1.00      0.69      0.81       147

    accuracy                           0.96      1115
   macro avg       0.98      0.84      0.90      1115
weighted avg       0.96      0.96      0.96      1115

[[968   0]
 [ 46 101]]


## Applying SVM

In [7]:
from sklearn import svm

#fitting SVM
clf = svm.SVC()
clf.fit(train_tfidf,y_train)
y_pred_svm = clf.predict(test_tfidf)

#accuracy score of SVM
score2 = metrics.accuracy_score(y_test, y_pred_svm)
print("The accuracy score of SVM is : ",score2)

#Confusion matrix of SVM

print(metrics.classification_report(y_test, y_pred_svm, target_names=['Ham', 'Spam']))
print(metrics.confusion_matrix(test_df["Label"], y_pred_svm))

The accuracy score of SVM is :  0.9847533632286996
              precision    recall  f1-score   support

         Ham       0.98      1.00      0.99       968
        Spam       1.00      0.88      0.94       147

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.99      0.98      0.98      1115

[[968   0]
 [ 17 130]]


#### Hyperparameter Tuning for SVM

In [8]:
from sklearn.model_selection import GridSearchCV
param_grid={"C":[0.1, 1, 10, 100], "kernel":("linear", "rbf")}
grid= GridSearchCV(clf, param_grid, refit=True, verbose=3)
grid.fit(train_tfidf,y_train)
y_pred_grid = grid.predict(val_tfidf)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ..............C=0.1, kernel=linear;, score=0.897 total time=   0.2s
[CV 2/5] END ..............C=0.1, kernel=linear;, score=0.895 total time=   0.2s
[CV 3/5] END ..............C=0.1, kernel=linear;, score=0.894 total time=   0.2s
[CV 4/5] END ..............C=0.1, kernel=linear;, score=0.894 total time=   0.2s
[CV 5/5] END ..............C=0.1, kernel=linear;, score=0.895 total time=   0.2s
[CV 1/5] END .................C=0.1, kernel=rbf;, score=0.857 total time=   0.4s
[CV 2/5] END .................C=0.1, kernel=rbf;, score=0.857 total time=   0.4s
[CV 3/5] END .................C=0.1, kernel=rbf;, score=0.858 total time=   0.4s
[CV 4/5] END .................C=0.1, kernel=rbf;, score=0.858 total time=   0.4s
[CV 5/5] END .................C=0.1, kernel=rbf;, score=0.858 total time=   0.4s
[CV 1/5] END ................C=1, kernel=linear;, score=0.966 total time=   0.2s
[CV 2/5] END ................C=1, kernel=linear;,

In [9]:
#accuracy score of SVM
score2 = metrics.accuracy_score(y_val, y_pred_grid)
print("The accuracy score of SVM is : ",score2)

#Confusion matrix of SVM

print(metrics.classification_report(y_val, y_pred_grid, target_names=['Ham', 'Spam']))
print(metrics.confusion_matrix(val_df["Label"], y_pred_grid))

The accuracy score of SVM is :  0.9928251121076234
              precision    recall  f1-score   support

         Ham       0.99      1.00      1.00       992
        Spam       0.99      0.94      0.97       123

    accuracy                           0.99      1115
   macro avg       0.99      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115

[[991   1]
 [  7 116]]


In [10]:
grid.best_params_

{'C': 10, 'kernel': 'linear'}

In [11]:
#fitting SVM after hyperparameter tuning
clf = svm.SVC(C=10, kernel ='linear')
clf.fit(train_tfidf,y_train)
y_pred_svm = clf.predict(test_tfidf)

#accuracy score of SVM
score2 = metrics.accuracy_score(y_test, y_pred_svm)
print("The accuracy score of SVM is : ",score2)

#Confusion matrix of SVM

print(metrics.classification_report(y_test, y_pred_svm, target_names=['Ham', 'Spam']))
print(metrics.confusion_matrix(test_df["Label"], y_pred_svm))

The accuracy score of SVM is :  0.9910313901345291
              precision    recall  f1-score   support

         Ham       0.99      1.00      0.99       968
        Spam       0.98      0.95      0.97       147

    accuracy                           0.99      1115
   macro avg       0.99      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115

[[965   3]
 [  7 140]]


## Neural Network

In [12]:
#Importing the required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional

In [13]:
# Defining pre-processing hyperparameters
max_len = 50 
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>" 
vocab_size = 500
embeding_dim = 16
n_dense = 24

#Tokenizing training data
tokenizer = Tokenizer(500)
tokenizer.fit_on_texts(X_train)

In [14]:
# Sequencing and padding on training and testing 
train_seq = tokenizer.texts_to_sequences(X_train)
train_pad = pad_sequences(train_seq, maxlen = max_len)
val_seq = tokenizer.texts_to_sequences(X_val)
val_pad = pad_sequences(val_seq, maxlen = max_len)
test_seq = tokenizer.texts_to_sequences(X_test)
test_pad = pad_sequences(test_seq, maxlen = max_len)

In [15]:
#Creating the neural network
model = Sequential()
model.add(Embedding(vocab_size, embeding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(24, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [16]:
# fitting the model to our data
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])
fit_model = model.fit(train_pad, y_train, epochs=30, validation_data=(val_pad, y_val))

Epoch 1/30
105/105 [==============================] - 1s 3ms/step - loss: 0.5397 - accuracy: 0.8573 - val_loss: 0.3483 - val_accuracy: 0.8897
Epoch 2/30
105/105 [==============================] - 0s 2ms/step - loss: 0.3593 - accuracy: 0.8573 - val_loss: 0.2884 - val_accuracy: 0.8897
Epoch 3/30
105/105 [==============================] - 0s 2ms/step - loss: 0.3032 - accuracy: 0.8588 - val_loss: 0.2206 - val_accuracy: 0.8987
Epoch 4/30
105/105 [==============================] - 0s 2ms/step - loss: 0.2014 - accuracy: 0.9159 - val_loss: 0.1355 - val_accuracy: 0.9659
Epoch 5/30
105/105 [==============================] - 0s 2ms/step - loss: 0.1269 - accuracy: 0.9587 - val_loss: 0.0866 - val_accuracy: 0.9776
Epoch 6/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0927 - accuracy: 0.9719 - val_loss: 0.0682 - val_accuracy: 0.9830
Epoch 7/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0760 - accuracy: 0.9779 - val_loss: 0.0573 - val_accuracy: 0.9839
Epoch 

In [17]:
# Model performance on validation data 
model.evaluate(val_pad, y_val)

35/35 [==============================] - 0s 1ms/step - loss: 0.0502 - accuracy: 0.9892


[0.05018720403313637, 0.9892376661300659]

#### Hyperparameter tuning for Neural Network

In [18]:
#Changing the loss function
# fitting the model on our data
model.compile(loss='binary_crossentropy',optimizer='SGD' ,metrics=['accuracy'])
fit_model = model.fit(train_pad, y_train, epochs=30, validation_data=(val_pad, y_val))

Epoch 1/30
105/105 [==============================] - 1s 3ms/step - loss: 0.0146 - accuracy: 0.9958 - val_loss: 0.0512 - val_accuracy: 0.9883
Epoch 2/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0147 - accuracy: 0.9964 - val_loss: 0.0562 - val_accuracy: 0.9857
Epoch 3/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0150 - accuracy: 0.9961 - val_loss: 0.0533 - val_accuracy: 0.9865
Epoch 4/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0146 - accuracy: 0.9961 - val_loss: 0.0511 - val_accuracy: 0.9883
Epoch 5/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0142 - accuracy: 0.9967 - val_loss: 0.0501 - val_accuracy: 0.9901
Epoch 6/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0145 - accuracy: 0.9961 - val_loss: 0.0508 - val_accuracy: 0.9883
Epoch 7/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0151 - accuracy: 0.9955 - val_loss: 0.0522 - val_accuracy: 0.9874
Epoch 

In [19]:
# Model performance on validation data 
model.evaluate(val_pad, y_val)

35/35 [==============================] - 0s 1ms/step - loss: 0.0525 - accuracy: 0.9874


[0.05253727734088898, 0.9874439239501953]

In [20]:
# Changing the number of epochs

# fitting a dense spam detector model
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])
fit_model = model.fit(train_pad, y_train, epochs=50, validation_data=(val_pad, y_val))
# Model performance on validation data 
model.evaluate(val_pad, y_val)

Epoch 1/50
105/105 [==============================] - 1s 2ms/step - loss: 0.0152 - accuracy: 0.9958 - val_loss: 0.0538 - val_accuracy: 0.9865
Epoch 2/50
105/105 [==============================] - 0s 2ms/step - loss: 0.0133 - accuracy: 0.9967 - val_loss: 0.0543 - val_accuracy: 0.9874
Epoch 3/50
105/105 [==============================] - 0s 2ms/step - loss: 0.0134 - accuracy: 0.9964 - val_loss: 0.0549 - val_accuracy: 0.9874
Epoch 4/50
105/105 [==============================] - 0s 2ms/step - loss: 0.0127 - accuracy: 0.9967 - val_loss: 0.0578 - val_accuracy: 0.9865
Epoch 5/50
105/105 [==============================] - 0s 2ms/step - loss: 0.0114 - accuracy: 0.9985 - val_loss: 0.0579 - val_accuracy: 0.9874
Epoch 6/50
105/105 [==============================] - 0s 2ms/step - loss: 0.0106 - accuracy: 0.9979 - val_loss: 0.0601 - val_accuracy: 0.9865
Epoch 7/50
105/105 [==============================] - 0s 2ms/step - loss: 0.0098 - accuracy: 0.9985 - val_loss: 0.0603 - val_accuracy: 0.9874
Epoch 

[0.09667690098285675, 0.9847533702850342]

As we can see the best accuracy is observed for Adam and number of epochs 30, so we predict the model on the test data.

In [21]:
# fitting the model 
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])
fit_model = model.fit(train_pad, y_train, epochs=30, validation_data=(test_pad, y_test))

# Model performance on test data 
results = model.evaluate(test_pad, y_test)

Epoch 1/30
105/105 [==============================] - 1s 2ms/step - loss: 0.0031 - accuracy: 0.9997 - val_loss: 0.0687 - val_accuracy: 0.9874
Epoch 2/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0032 - accuracy: 0.9997 - val_loss: 0.0696 - val_accuracy: 0.9874
Epoch 3/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 0.9997 - val_loss: 0.0699 - val_accuracy: 0.9857
Epoch 4/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 0.9997 - val_loss: 0.0711 - val_accuracy: 0.9865
Epoch 5/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0032 - accuracy: 0.9997 - val_loss: 0.0711 - val_accuracy: 0.9857
Epoch 6/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0028 - accuracy: 0.9997 - val_loss: 0.0844 - val_accuracy: 0.9812
Epoch 7/30
105/105 [==============================] - 0s 2ms/step - loss: 0.0037 - accuracy: 0.9997 - val_loss: 0.0711 - val_accuracy: 0.9865
Epoch 

In [23]:
print("Neural Network gives us an accuracy of :", results[1])

Neural Network gives us an accuracy of : 0.9865471124649048
